In [1]:
# -*- coding: utf-8 -*-

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, roc_curve,roc_auc_score,classification_report
from sklearn import (metrics, cross_validation, linear_model, preprocessing)

'''import seaborn as sns'''
from time import time
# Making the splits
y = pd.read_csv('/home/w/Data Mining/HW1/amazon/train.csv')
X = pd.read_csv('/home/w/Data Mining/HW1/amazon/train.csv')
y_test = pd.read_csv('/home/w/Data Mining/HW1/amazon/test.csv')
X_test = pd.read_csv('/home/w/Data Mining/HW1/amazon/test.csv')

gb = y.groupby("ACTION")

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
gb.size()

ACTION
0     1897
1    30872
dtype: int64

In [ ]:
# === one-hot encoding === #
    # we want to encode the category IDs encountered both in
    # the training and the test set, so we fit the encoder on both
encoder = preprocessing.OneHotEncoder()
encoder.fit(np.vstack((X, X_test)))
X = encoder.transform(X)  # Returns a sparse matrix (see numpy.sparse)
X_test = encoder.transform(X_test)

In [3]:
X.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [4]:
y = y.ACTION

In [5]:
X = X.iloc[:,1:]

In [6]:
X.head()

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [8]:
y.head()
y.count()

32769

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [12]:
y_test.head()

18654    1
19639    1
24971    1
18359    1
11252    1
Name: ACTION, dtype: int64

In [4]:
# Feature scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)
y_train = sc.fit_transform(y_train)
y_test = sc.fit_transform(y_test)

In [10]:
y_train

array([[-4.01630359,  1.03589459, -0.55371817, ...,  2.03612968,
        -0.64826938, -0.25547904],
       [ 0.24898516,  0.91392597, -0.23446893, ..., -0.75066971,
         0.86437959, -0.30580058],
       [ 0.24898516,  0.83578892, -0.02800481, ..., -0.72147409,
         0.65690321, -0.27307398],
       ..., 
       [ 0.24898516, -0.78888613, -0.29422821, ...,  1.66879469,
         1.08614172, -0.13847266],
       [ 0.24898516, -0.0448142 ,  1.27457923, ..., -0.75265639,
         1.06749171, -0.25723853],
       [ 0.24898516, -0.0448142 , -0.26092574, ...,  2.02153187,
        -0.64826938, -0.2215208 ]])

In [13]:
#RandomForest
from sklearn.ensemble import RandomForestClassifier
 
rf = RandomForestClassifier(criterion='gini', 
                             n_estimators=80,
                             min_samples_split=12,
                             min_samples_leaf=1,
                             oob_score=True,
                             random_state=1,
                             n_jobs=-1) 

In [40]:
#SVM
from sklearn.svm import SVC
svm_clf = SVC(probability=True)

In [41]:
#SVM
svm_clf.fit(X_train,y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [42]:
#SVM
svmpre = svm_clf.predict_proba(X_test)[:,1]

fpr2, tpr2, thresholds2 = metrics.roc_curve(y_test, svmpre)
roc_auc2 = metrics.auc(fpr2, tpr2)
print "svm AUC: %f" % (roc_auc2)

svm AUC: 0.532557


In [20]:
rf.fit(X_train,y_train)
preds = rf.predict_proba(X_test)[:, 1]
# compute AUC metric for this CV fold
fpr, tpr, thresholds = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)
print "AUC: %f" % (roc_auc)

AUC: 0.854420


In [33]:
# 利用模型進行預測
testset = pd.read_csv('/home/w/Data Mining/HW1/amazon/test.csv')
rf.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=12,
            min_weight_fraction_leaf=0.0, n_estimators=80, n_jobs=-1,
            oob_score=True, random_state=1, verbose=0, warm_start=False)

In [34]:
testset2 = testset.iloc[:,1:]
testset2.head(5)

,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,78766,72734,118079,118080,117878,117879,118177,19721,117880
1,40644,4378,117961,118327,118507,118863,122008,118398,118865
2,75443,2395,117961,118300,119488,118172,301534,249618,118175
3,43219,19986,117961,118225,118403,120773,136187,118960,120774
4,42093,50015,117961,118343,119598,118422,300136,118424,118425


In [30]:
preds = rf.predict_proba(testset2)[:,1]

In [31]:
preds

array([ 0.91325584,  0.99852941,  0.99337798, ...,  0.95390346,
        0.86891587,  1.        ])

In [32]:
submit = pd.read_csv('/home/w/Data Mining/HW1/amazon/sampleSubmission.csv')

submit['Action'] = preds

submit.to_csv('/home/w/Data Mining/HW1/amazon/test02.csv', index= False)

#submission = pd.DataFrame({"PassengerId": titanic_test["PassengerId"], "Survived": svm_output })
#submission.to_csv("/home/w/Desktop/tmp/SVMv1.csv", index=False) 

In [11]:
preds.size

58921